# Demonstrate Workflow Runner for OGC Application Packages
## This notebook runs through some example API calls to the Workflow Runner on the EODH Platform which is build on the ADES (Application, Deployment Execution Service) component from EOEPCA

In [1]:
!pip install urllib3
!pip install pillow
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [2]:
import json
import time
import urllib3
http = urllib3.PoolManager(cert_reqs='CERT_NONE')
urllib3.disable_warnings() ## to avoid SSL warnings

In [3]:
## Define text colour for later output
class bcolors:
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    ENDC = '\033[0m'

In [4]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file (if present)
load_dotenv("sample.env", override=True)

api_token = os.getenv("TOKEN")

In [ ]:
## Place your workspace-scoped API token here
workspace = "<YOUR-WORKSPACE-HERE>" # must align with the workspace-scoped token used above
auth_dict = {"Authorization": f"Bearer {api_token}"}

## Below are some example API requests you can make to the ADES component
Feel free to run these examples and change the inputs by specifying the application packages, process name and process inputs.

As an example we provide three EOEPCA-developed OGC Application Package to demonstrate the successful execution using the ADES deployment:
- [convert-url](https://github.com/EOEPCA/convert/blob/main/convert-url-app.cwl) - take an image specified by a URL and resize it by a given scale percentage
- [convert-stac](https://github.com/EOEPCA/convert/blob/main/convert-stac-app.cwl) - take an image specified by a stac item and resize it by a given scale percentage
- [water-bodies](https://github.com/EOEPCA/deployment-guide/blob/main/deploy/samples/requests/processing/water-bodies-app.cwl) - takes STAC items, area of interest, epsg definition and set of bands and identifies water bodies based on NDWI and Otsu threshold

This application is specified by configuring the below variable

In [6]:
# Update these variables as required to identify the running ades instance and specify workspace name
# If the workspace does not yet exect, it will be created by the ades automatically
wr_endpoint = "eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs"

# Configure workflow name and inputs
workflow_id = "snuggs"
# Open this link to view the file
cwl_location = "https://raw.githubusercontent.com/EO-DataHub/eodhp-ades-demonstration/refs/heads/main/snuggs.cwl"
inputs_dict = {
                "inputs": {
                    "input_reference": [
                        "https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A"
                    ],
                    "s_expression": "ndvi: (/ (- B08 B04) (+ B08 B04))" 
                    }
              }
print(f"CWL is located at {cwl_location}")
print(f"With Application Package code available at https://github.com/EOEPCA/app-snuggs/tree/main/src/s_expression")

CWL is located at https://raw.githubusercontent.com/EO-DataHub/eodhp-ades-demonstration/refs/heads/main/snuggs.cwl
With Application Package code available at https://github.com/EOEPCA/app-snuggs/tree/main/src/s_expression


### List processes

In [9]:
url = f"https://{wr_endpoint}/{workspace}/processes"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'processes': [{'id': 'echo',
   'title': 'Echo input',
   'description': 'Simply echo the value provided as input',
   'mutable': False,
   'version': '2.0.0',
   'metadata': [{'title': 'Demo'}],
   'jobControlOptions': ['sync-execute', 'async-execute', 'dismiss'],
   'outputTransmission': ['value', 'reference'],
   'links': [{'rel': 'self',
     'type': 'application/json',
     'title': 'Process Description',
     'href': 'https://eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs/tjellicoetpzuk/processes/echo'}],
   'AccessReasoning': 'Workspace owns this workflow'}],
 'links': [{'rel': 'self',
   'type': 'application/json',
   'href': 'https://eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs/tjellicoetpzuk/processes'}]}

### Undeploy/Delete process

In [8]:
## Here a 204 response means the process was remove successfully, no other content is returned
url = f"https://{wr_endpoint}/{workspace}/processes/{workflow_id}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('DELETE', url, headers=headers)
response.status

204

### Deploy processes

In [10]:
url = f"https://{wr_endpoint}/{workspace}/processes"
headers = {"Accept": "application/json", "Content-Type": "application/json"}
headers.update(auth_dict)
params = {"executionUnit": {
            "href": f"{cwl_location}",
            "type": "application/cwl"
            }
         }
response = http.request('POST', url, headers=headers, body=json.dumps(params))
deployStatus = response.headers['Location']
json.loads(response.data)

{'id': 'snuggs',
 'title': 's expressions',
 'description': 'Applies s expressions to EO acquisitions',
 'mutable': True,
 'version': '0.3.0',
 'metadata': [{'role': 'https://schema.org/softwareVersion',
   'value': '0.3.0'}],
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs/tjellicoetpzuk/processes/snuggs/execution'}]}

### Get process details

In [12]:
# This is where another user might come to discover more about the workflow
url = f"https://{wr_endpoint}/{workspace}/processes/{workflow_id}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'id': 'snuggs',
 'title': 's expressions',
 'description': 'Applies s expressions to EO acquisitions',
 'mutable': True,
 'version': '0.3.0',
 'metadata': [{'role': 'https://schema.org/softwareVersion',
   'value': '0.3.0'}],
 'outputTransmission': ['value', 'reference'],
 'jobControlOptions': ['async-execute', 'dismiss'],
 'links': [{'rel': 'http://www.opengis.net/def/rel/ogc/1.0/execute',
   'type': 'application/json',
   'title': 'Execute End Point',
   'href': 'https://eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs/tjellicoetpzuk/processes/snuggs/execution'}],
 'inputs': {'input_reference': {'title': 'Input product reference',
   'description': 'Input product reference',
   'maxOccurs': 999,
   'extended-schema': {'type': 'array',
    'items': {'type': 'string'},
    'minItems': 1,
    'maxItems': 999},
   'schema': {'type': 'string'}},
  's_expression': {'title': 's expression',
   'description': 's expression',
   'maxOccurs': 999,
   'extended-schema': {'type': 'arr

### Execute process

In [13]:
url = f"https://{wr_endpoint}/{workspace}/processes/{workflow_id}/execution"
headers = {"Accept": "application/json", "Content-Type": "application/json", "Prefer": "respond-async"}
headers.update(auth_dict)
params = {**inputs_dict}
response = http.request('POST', url, headers=headers, body=json.dumps(params))
executeStatus = response.headers['Location']
response_json = json.loads(response.data)
job_id = response_json["id"]
print(f"Running under jobID {job_id}")

Running under jobID 5c8c1646-3615-11f0-88b3-56fda0cd4f2f


### Get execute status
See the following section to continually poll this function instead to determine once complete

In [14]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
time.sleep(5)
response = http.request('GET', url, headers=headers)
json.loads(response.data)

{'progress': 20,
 'id': '5c8c1646-3615-11f0-88b3-56fda0cd4f2f',
 'jobID': '5c8c1646-3615-11f0-88b3-56fda0cd4f2f',
 'type': 'process',
 'processID': 'snuggs',
 'created': '2025-05-21T07:29:42.804Z',
 'started': '2025-05-21T07:29:42.804Z',
 'updated': '2025-05-21T07:29:48.906Z',
 'status': 'running',
 'message': 'upload required files',
 'links': [{'title': 'Status location',
   'rel': 'monitor',
   'type': 'application/json',
   'href': 'https://eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs/tjellicoetpzuk/jobs/5c8c1646-3615-11f0-88b3-56fda0cd4f2f'}]}

### Get execute status (continuous polling)
Run this cell to keep polling the ExecuteStatus endpoint to determine when the process has finished running and also see it's final status: *SUCCESS* or *FAILED*

In [15]:
url = f"{executeStatus}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
data = json.loads(response.data)
status = data['status']
message = data['message']
print("Status is " + bcolors.OKBLUE + status.upper() + bcolors.ENDC)
print("Message is " + "\033[1m" + message + "\033[0m", end="")
old_message = message
old_status = status
while status == "running":
    time.sleep(2)
    response = http.request('GET', url, headers=headers)
    data = json.loads(response.data)
    status = data['status']
    message = data['message']
    if status != old_status:
        print("\n")
        print("Status is " + bcolors.OKBLUE + status.upper() + bcolors.ENDC)
        print("Message is " + "\033[1m" + message + "\033[0m", end="")
    elif message != old_message:
        print(".")
        print("Message is " + "\033[1m" + message + "\033[0m", end="")
    else:
        print(".", end="")
    old_message = message
    old_status = status

if status == "successful":
    print("\n")
    print(bcolors.OKGREEN + "SUCCESS" + bcolors.ENDC)

if status == "failed":
    print(bcolors.WARNING + "FAILED" + bcolors.ENDC)

Status is RUNNING
Message is upload required files...
Message is execution submitted.......................................
Message is delivering outputs, logs and usage report...

Status is SUCCESSFUL
Message is ZOO-Kernel successfully run your service!

SUCCESS


### Get processing results

In [16]:
## Note, this will return a 500 response when no output is produced
url = f"{executeStatus}/results"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
response_json = json.loads(response.data)
response_json

{'type': 'Collection',
 'id': 'col_5c8c1646-3615-11f0-88b3-56fda0cd4f2f',
 'stac_version': '1.0.0',
 'description': 'description',
 'links': [{'rel': 'root',
   'href': 'https://tjellicoetpzuk.eodatahub-workspaces.org.uk/files/workspaces-eodhp/processing-results/snuggs.json',
   'type': 'application/json',
   'title': 'snuggs Outputs Catalog'},
  {'rel': 'item',
   'href': 'https://tjellicoetpzuk.eodatahub-workspaces.org.uk/files/workspaces-eodhp/processing-results/snuggs/catalog/col_5c8c1646-3615-11f0-88b3-56fda0cd4f2f/S2B_53HPA_20210723_0_L2A.json',
   'type': 'application/json'},
  {'rel': 'self',
   'href': 'https://tjellicoetpzuk.eodatahub-workspaces.org.uk/files/workspaces-eodhp/processing-results/snuggs/catalog/col_5c8c1646-3615-11f0-88b3-56fda0cd4f2f.json',
   'type': 'application/json'},
  {'rel': 'parent',
   'href': 'https://tjellicoetpzuk.eodatahub-workspaces.org.uk/files/workspaces-eodhp/processing-results/snuggs/catalog.json',
   'type': 'application/json'}],
 'title': 'R

### List jobs

In [56]:
url = f"https://{wr_endpoint}/{workspace}/jobs"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
response = http.request('GET', url, headers=headers)
# json.loads(response.data)

### Delete a Running Job

In [57]:
job_id = "your-job-id"
url = f"https://{wr_endpoint}/{workspace}/jobs/{job_id}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
## Uncomment the following lines if you wish to delete the running Job as specified in `job_id` above
# response = http.request('DELETE', url, headers=headers)
# response.status

### Undeploy/Delete process

In [58]:
## Here a 204 response means the process was remove successfully, no other content is returned
url = f"https://{wr_endpoint}/{workspace}/processes/{workflow_id}"
headers = {"Accept": "application/json"}
headers.update(auth_dict)
params = {}
# response = http.request('DELETE', url, headers=headers)
# response.status


### View Resulting Data in the Resource Catalogue

In [19]:
## View resulting dataset
results_catalog_url = f"https://eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs/{workspace}/catalogs/processing-results/catalogs/{workflow_id}"
results_items_url = results_catalog_url + f"/catalogs/catalog/collections/col_{job_id}/items"
print(f"Looking for items at {results_items_url}")
response = http.request('GET', results_items_url, headers=headers)

items = json.loads(response.data)
print(json.dumps(items, indent=2))

Looking for items at https://eodatahub.org.uk/api/catalogue/stac/catalogs/user/catalogs/tjellicoetpzuk/catalogs/processing-results/catalogs/snuggs/catalogs/catalog/collections/col_5c8c1646-3615-11f0-88b3-56fda0cd4f2f/items
{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "stac_version": "1.0.0",
      "stac_extensions": [
        "https://stac-extensions.github.io/eo/v1.1.0/schema.json",
        "https://stac-extensions.github.io/view/v1.0.0/schema.json",
        "https://stac-extensions.github.io/projection/v1.1.0/schema.json"
      ],
      "id": "S2B_53HPA_20210723_0_L2A",
      "collection": "col_5c8c1646-3615-11f0-88b3-56fda0cd4f2f",
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              136.11273785955868,
              -36.22788818051635
            ],
            [
              136.09905192261127,
              -35.238096451039816
            ],
            [
              137.3051346

In [20]:
# Extract first item
item = items["features"][0]

# Extract assets
assets = item["assets"]
print(json.dumps(assets, indent=2))


{
  "ndvi": {
    "href": "https://tjellicoetpzuk.eodatahub-workspaces.org.uk/files/workspaces-eodhp/processing-results/snuggs/catalog/col_5c8c1646-3615-11f0-88b3-56fda0cd4f2f/S2B_53HPA_20210723_0_L2A/ndvi.tif",
    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
    "roles": [
      "data"
    ],
    "s-expression": " (/ (- B08 B04) (+ B08 B04))"
  }
}


In [ ]:
## Download asset tif
asset_href = assets["ndvi"]["href"]

import requests
from PIL import Image
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Fetch the file from the URL
response = requests.get(asset_href, headers=headers)
response.raise_for_status()  # Ensure the request was successful

# Open the image using Pillow
image = Image.open(io.BytesIO(response.content))
data = np.array(image)  # Convert image to a numpy array

# Convert the numpy array to a dataframe
df = pd.DataFrame(data)
df

/home/tjellicoe/.local/lib/python3.9/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/tmp/ipykernel_3139010/4029050315.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# Plotting the heatmap
plt.imshow(data, cmap='viridis')
plt.colorbar()
plt.title("Normalised Difference Vegetation Index")
plt.show()